<a href="https://colab.research.google.com/github/rhuntsman/DeepLearning2025/blob/main/Depth_v_Breadth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from IPython import display
display.set_matplotlib_formats('svg')
import seaborn as sns


<ipython-input-22-9057d05cc560>:7: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  display.set_matplotlib_formats('svg')


In [25]:
iris = sns.load_dataset('iris')

data = torch.tensor(iris[iris.columns[0:4]].values).float()

In [53]:
labels = torch.zeros(len(data), dtype=torch.long)

labels[iris.species=='versicolor'] = 1
labels[iris.species=='virginica'] = 2

labels

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

In [54]:
class ANNIris(nn.Module):
  def __init__(self, nUnits, nLayers):
    super().__init__()

    self.layers = nn.ModuleDict()
    self.nLayers = nLayers

    self.layers['input'] = nn.Linear(4,nUnits)

    for i in range(nLayers):
      self.layers[f'hidden{i}'] = nn.Linear(nUnits, nUnits)

    self.layers['output'] = nn.Linear(nUnits, 3)

    # forward pass

  def forward(self, x):
    x = self.layers['input'](x)

  # hidden layers
    for i in range(self.nLayers):
      x = F.relu( self.layers[f'hidden{i}'](x) )

  # return output layer
    x = self.layers['output'](x)
    return x

In [55]:
nUnitsPerLayer = 12
nLayers = 4
net = ANNIris(nUnitsPerLayer, nLayers)
net

ANNIris(
  (layers): ModuleDict(
    (input): Linear(in_features=4, out_features=12, bias=True)
    (hidden0): Linear(in_features=12, out_features=12, bias=True)
    (hidden1): Linear(in_features=12, out_features=12, bias=True)
    (hidden2): Linear(in_features=12, out_features=12, bias=True)
    (hidden3): Linear(in_features=12, out_features=12, bias=True)
    (output): Linear(in_features=12, out_features=3, bias=True)
  )
)

In [56]:
test = torch.randn(10,4)

y = net(test)

print( y.shape ), print(' ')

print(y)

torch.Size([10, 3])
 
tensor([[-0.0691, -0.1221, -0.2899],
        [-0.0747, -0.1243, -0.2900],
        [-0.0722, -0.1273, -0.2922],
        [-0.0673, -0.1336, -0.2837],
        [-0.0723, -0.1286, -0.2821],
        [-0.0621, -0.1318, -0.2748],
        [-0.0739, -0.1269, -0.2848],
        [-0.0771, -0.1246, -0.2829],
        [-0.0710, -0.1301, -0.2906],
        [-0.0705, -0.1219, -0.2888]], grad_fn=<AddmmBackward0>)


In [58]:
def trainModel(model):

  lossFunction = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

  for epoch_i in range(nEpochs):

    # make predictions
    y_hat = model(data)

    # calculate the loss
    loss = lossFunction(y_hat, labels)

    # backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


  predictions = model(data)
  predLabels = torch.argmax(predictions, axis=1)
  acc = torch.mean((predLabels == labels).float())

  nParams = sum(p.numel() for p in model.parameters() if p.requires_grad)

  return acc, nParams



In [60]:
# Test model
nEpochs = 2500
acc = trainModel(net)

print(acc)

(tensor(0.9800), 723)


In [62]:
# define model paramters
n_layers = range(1,6)
n_units = range(4,101,3)

# initialize output matrices
accuracies = np.zeros((len(n_units), len(n_layers)))
total_parameters = np.zeros((len(n_units), len(n_layers)))

n_epochs = 500

for unit_idx in range(len(n_units)):
  for layer_idx in range(len(n_layers)):
    net = ANNIris(n_units[unit_idx], n_layers[layer_idx])

    acc, n_parameters = trainModel(net)
    accuracies[unit_idx, layer_idx] = acc
    total_parameters[unit_idx, layer_idx] = n_parameters




In [65]:
total_parameters

array([[   55.,    75.,    95.,   115.,   135.],
       [  115.,   171.,   227.,   283.,   339.],
       [  193.,   303.,   413.,   523.,   633.],
       [  289.,   471.,   653.,   835.,  1017.],
       [  403.,   675.,   947.,  1219.,  1491.],
       [  535.,   915.,  1295.,  1675.,  2055.],
       [  685.,  1191.,  1697.,  2203.,  2709.],
       [  853.,  1503.,  2153.,  2803.,  3453.],
       [ 1039.,  1851.,  2663.,  3475.,  4287.],
       [ 1243.,  2235.,  3227.,  4219.,  5211.],
       [ 1465.,  2655.,  3845.,  5035.,  6225.],
       [ 1705.,  3111.,  4517.,  5923.,  7329.],
       [ 1963.,  3603.,  5243.,  6883.,  8523.],
       [ 2239.,  4131.,  6023.,  7915.,  9807.],
       [ 2533.,  4695.,  6857.,  9019., 11181.],
       [ 2845.,  5295.,  7745., 10195., 12645.],
       [ 3175.,  5931.,  8687., 11443., 14199.],
       [ 3523.,  6603.,  9683., 12763., 15843.],
       [ 3889.,  7311., 10733., 14155., 17577.],
       [ 4273.,  8055., 11837., 15619., 19401.],
       [ 4675.,  883